# Fine Tuning Mini Project

In [8]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import certifi
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [9]:
import ssl, certifi
ssl._create_default_https_context = lambda: ssl.create_default_context(cafile=certifi.where())


In [10]:
# Load the CIFAR-10 dataset

(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [11]:
# Normalize the pixel values to [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

print(X_train.dtype, X_train.min(), X_train.max(), X_train.shape)
print(X_test.dtype, X_test.min(), X_test.max(), X_test.shape)

float32 0.0 1.0 (50000, 32, 32, 3)
float32 0.0 1.0 (10000, 32, 32, 3)


In [12]:
# One-hot encode the labels
y_train = to_categorical(y_train.squeeze(), num_classes=10)
y_test = to_categorical(y_test.squeeze(), num_classes=10)

print(y_train.shape, y_test.shape)   # (50000, 10) (10000, 10)
print(y_train.dtype)                 # float32
print(y_train[0], y_train[0].sum()) 

(50000, 10) (10000, 10)
float64
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.] 1.0


In [13]:
# Split the data into training and validation sets

class_ids = np.argmax(y_train, axis=1)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, 
    test_size=0.1, 
    random_state=8, 
    stratify=class_ids
)

print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)

(45000, 32, 32, 3) (5000, 32, 32, 3)
(45000, 10) (5000, 10)


In [14]:
# Load the pre-trained VGG16 model (excluding the top classifier)

base_model = VGG16(weights="imagenet", include_top=False, input_shape=(32, 32, 3))

2025-11-10 22:22:54.806407: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-11-10 22:22:54.806453: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-11-10 22:22:54.806468: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-11-10 22:22:54.806483: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-10 22:22:54.806494: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [15]:
# Freeze the layers in the base model
base_model.trainable = False

In [16]:
# Add a global average pooling layer

x = base_model.output
x = GlobalAveragePooling2D()(x)

In [17]:
# Add a fully connected layer with 256 units and ReLU activation
x = Dense(256, activation="relu")(x)

In [18]:
# Add the final classification layer with 10 units (for CIFAR-10 classes) and softmax activation
outputs = Dense(10, activation="softmax")(x)

In [19]:
# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=outputs)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,586 (56.64 MB)

 Trainable params: 133,898 (523.04 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [20]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001), 
    loss="categorical_crossentropy", 
    metrics=["accuracy"]
)

In [21]:
# Train the model
callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=256,
    callbacks=[callback],
    verbose=1
)

Epoch 1/10


2025-11-10 22:23:02.628458: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


176/176 ━━━━━━━━━━━━━━━━━━━━ 19s 86ms/step - accuracy: 0.4649 - loss: 1.5395 - val_accuracy: 0.5392 - val_loss: 1.3245
Epoch 2/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 15s 84ms/step - accuracy: 0.5606 - loss: 1.2685 - val_accuracy: 0.5702 - val_loss: 1.2414
Epoch 3/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 14s 81ms/step - accuracy: 0.5818 - loss: 1.2124 - val_accuracy: 0.5738 - val_loss: 1.2318
Epoch 4/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 14s 81ms/step - accuracy: 0.5902 - loss: 1.1820 - val_accuracy: 0.5756 - val_loss: 1.2154
Epoch 5/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 15s 85ms/step - accuracy: 0.5968 - loss: 1.1644 - val_accuracy: 0.5780 - val_loss: 1.2062
Epoch 6/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 15s 85ms/step - accuracy: 0.6010 - loss: 1.1536 - val_accuracy: 0.5824 - val_loss: 1.2144
Epoch 7/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 14s 81ms/step - accuracy: 0.6032 - loss: 1.1469 - val_accuracy: 0.5888 - val_loss: 1.1993
Epoch 8/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 15s 83ms/step - accuracy: 0.6059 - loss: 1.1410 - val_accurac

In [22]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.5824 - loss: 1.2046
Test Loss: 1.2046
Test Accuracy: 0.5824
